# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** I think this is a Classification prediction. The objective is to find students that will fail or not. This is even a binary classification (two excludents labels, i.e. "yes" or "no"). There is neither continuous or numerical (common values for regression prediction) values for the possibles Y's.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [15]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len([h for h in student_data.columns if h != 'passed'])

# TODO: Calculate passing students
n_passed = len(student_data[student_data.passed == 'yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data.passed == 'no'])

# TODO: Calculate graduation rate
grad_rate = n_passed*100.0/n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [16]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [17]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [19]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=57)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **
#### KNN #### 
**Real-world application**.
A variance of KNN was used to help semiconductor manufacturers to fault detection [1]. KNN has advantages[2] like:
* Simplicity.
* Explanation of the classifier (i.e. effective in analysis of the neighbours).
* Techniques for Noise Reductions specific for KNN that improve accuracy[3].

In the other hand, there some disadvantages:
* Poor runtime performance when training set is large
* Great sensitivity to irrelevant or redundant features and, thus, to classification

To this project KNN is a good choice, I think, because the training set isn't too large and the metric for distance, or some similarity notion, wont cost to much computational power, once the most of the features are categorical.

#### Decision Trees ####
**Real-world application**. Predicting electricity energy consumption. Decision Tree and Neural Network proved to be a good alternative to regression analysis [4].

Decision Tree has the advantages[5]:
* Complex decision regions can be approximated by the agreggation of simpler decision locally
* A sample is tested against only certain subsets and not against all classes, eliminating unnecessary computations.
* Flexibility of choosing different subsets of features at internal node of the tree

As disadvantages[5]:
* Biased gain of information to features with much different number of values.
* Overlap, especially when the numver of classes is large, causing, probably, a much larger number of terminals then the actual number of classes, increasing the search time and memory space.
* Difficulties in designing and optimal DTC, once the performance strongly depends on how well the tree is designed.

Decision Tree is a good candidate, in my opinion, for this project because the features haven't greate values (most of it has few categories, even bynary) and the continuous one is in a small range. Other thing is that the output can be considered boolean. The features appears to complete with small quantity of noise, and no missing data. 

#### Support Vector Machine ####
**Real-world application**. Bankruptcy prediction model[6].

Advantages:
* Effective in high dimensional spaces.
* Effective when number of dimensions is greater than the number of samples.
* Versatile (i.e. domain knowledge can be used to create specific decision function). So both common kernels or specifics is possibles.

Disadvantages:
* If the number of features is much greater than the number of samples, gives poor performance.
* Do not directly provide estimates, to calculate it is necessary some computationally expensive artifacts

SVM were developed for binary classification. This and from what I understand, there is no need in generate some probability for chance of fault, but only if the student will fall or not, so SVM is a good candidate.

[1] He, Q. Peter, and Jin Wang. "Fault detection using the k-nearest neighbor rule for semiconductor manufacturing processes." IEEE Transactions on Semiconductor Manufacturing 20.4 (2007): 345-354.

[2] Cunningham, Padraig, and Sarah Jane Delany. "k-Nearest neighbour classifiers." Multiple Classifier Systems 34 (2007): 1-17.

[3] Delany, Sarah Jane, and Pádraig Cunningham. "An analysis of case-base editing in a spam filtering system." European Conference on Case-Based Reasoning. Springer Berlin Heidelberg, 2004.

[4] Tso, Geoffrey KF, and Kelvin KW Yau. "Predicting electricity energy consumption: A comparison of regression analysis, decision tree and neural networks." Energy 32.9 (2007): 1761-1768.

[5] Safavian, S. Rasoul, and David Landgrebe. "A survey of decision tree classifier methodology." IEEE transactions on systems, man, and cybernetics 21.3 (1991): 660-674.

[6] Shin, Kyung-Shik, Taik Soo Lee, and Hyun-jung Kim. "An application of support vector machines in bankruptcy prediction model." Expert Systems with Applications 28.1 (2005): 127-135.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [20]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [24]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A - KNN
from sklearn.neighbors import KNeighborsClassifier
# from sklearn import model_B - Decistion Tree
from sklearn import tree
# from sklearn import model_C - SVM
from sklearn.svm import SVC

# TODO: Initialize the three models
clf_A = KNeighborsClassifier()
clf_B = tree.DecisionTreeClassifier(random_state = 57)
clf_C = SVC(random_state = 57)

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test)

# TODO: Set up the training set sizes
X_train_100, y_train_100 = X_train[:100], y_train[:100]
X_train_200, y_train_200 = X_train[:200], y_train[:200]
X_train_300, y_train_300 = X_train[:300], y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# clf_A, 100
#train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
#print "----------------"

#clf_A, 200
#train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
#print "----------------"

#clf_A, 300
#train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
#print "----------------"

# clf_B, 100
#train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
#print "----------------"

#clf_B, 200
#train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
#print "----------------"

#clf_B, 300
#train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
#print "----------------"

# clf_C, 100
#train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
#print "----------------"

#clf_C, 200
#train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
#print "----------------"

#clf_C, 300
#train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0012 seconds
Made predictions in 0.0036 seconds.
F1 score for training set: 0.8322.
Made predictions in 0.0055 seconds.
F1 score for test set: 0.8514.
----------------
Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.0014 seconds
Made predictions in 0.0045 seconds.
F1 score for training set: 0.8223.
Made predictions in 0.0029 seconds.
F1 score for test set: 0.7971.
----------------
Training a KNeighborsClassifier using a training set size of 300. . .
Trained model in 0.0015 seconds
Made predictions in 0.0072 seconds.
F1 score for training set: 0.8604.
Made predictions in 0.0073 seconds.
F1 score for test set: 0.8116.
----------------
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0019 seconds
Made predictions in 0.0006 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0005 seconds.
F1 score for test 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - KNeighborsClassifier() **

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0017s          |        0.0037s         |     0.8917       |     0.7891      |
| 200               |        0.0016s          |        0.0057s         |     0.8707       |     0.7857      |
| 300               |        0.0022           |        0.0101s         |     0.8784       |     0.7164      |

** Classifer 2 - DecisionTreeClassifier**  

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0018s             |      0.0006s           |        1         |     0.6772      |
| 200               |     0.0029s             |      0.0006s           |        1         |     0.7887      |
| 300               |     0.0026s             |      0.0003s           |        1         |     0.7154      |

** Classifer 3 - svm.SVC**  

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0031s            |      0.0026s           |    0.8811        |    0.8163       |
| 200               |      0.0044s            |      0.0026s           |    0.8913        |    0.7943       |
| 300               |      0.0107s            |      0.0025s           |    0.9074        |    0.8169       |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** Tests have shown that the vector support machine had the best performance given the test data. Training set with 100 or 300 has, approximately, the same accuracy. SVM has the higher time to train, but still a low time considering 0.01 (# trainig 300) or o.003 (# trainig 300) seconds not much time to spend. So, for this reason I think that SVM could be the most approriate model to use in this prediction.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** (SVM) is a type of "Supervised Learning", wich is some stuff that uses historical data that has already been classified (in this case students that have fail or not) to classify new examples (tell if a new student will fail or not).
In SVM some lines (techically is "hyperplane", but do not worry about mathematical stuff) are created to separate the data, but mathematically there is infinite lines that can separate the data (see the imagem below from wikipedia):

![Alt][1]

[1]: https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Svm_separating_hyperplanes_%28SVG%29.svg/512px-Svm_separating_hyperplanes_%28SVG%29.svg.png "Example of lines that separate the data"
`You can see that lines H2 and H3 separates perfectly all the data, but intuitively, and mathematically H3 do it better.`

To select the best line (i.e the line that better separate and describe the data) the SVM maximize the distance beetwen the two lines that separates each class. This distance between the two is called "margin" and there shouldn't have any data between them. The line in the middel is the maximized line. See the image below that shows the better line and the margin to to visualize the argument (get the image from docs.opencv.org):

![Alt][2]

[2]: http://docs.opencv.org/2.4/_images/optimal-hyperplane.png "SVM with max separate hyperplane"

Some times the data can't be separated with lines just as is (e.g. polynomials). For these some transformations (e.g. get the distance to the origin) in the data can be made to transform it in some way that helps the SVM to create these lines that in original data can create some circles or other forms, for examples.

SVM is trained using the examples that generate some kind of functions that are capables, with some accuracy, to separate and classify new examples. In this case to predict students that will fail, based on the behavior of previous students and their relations and nuances. In short, given data with labels, SVM output the formula ("hyperplane") to classify new students.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [2]:
# the examples in sklearn for gridsearch and make_score use SVC(). This was coincidence.
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.svm import SVC

# TODO: Create the parameters list you wish to tune
# from the lib author, use first C 1 and 100 and then check intermediare values, if needed
# Using rbf and tuning its parameters
# Changind gamma to see what happens!
# Using the reviwer tips:
from sklearn.preprocessing import normalize
X_train = normalize(X_train)
X_test = normalize(X_test)

C_range = np.logspace(-3,3,13)
gamma_range = np.logspace(-4, 2, 13)
parameters = dict(gamma=gamma_range, C=C_range)
#parameters = {'C': [1, 10, 100, 1000], 'gamma': ['auto', 0.025, 0.075, 0.1, 0.125, 0.175, 0.2 ]}

#print "C_range: ", C_range
#print "gamma_range: ", gamma_range

# TODO: Initialize the classifier
clf = SVC(random_state=57)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
# using the function to get training time
train_classifier(grid_obj, X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

NameError: name 'X_train' is not defined

### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** Both F1 score for training and test was better then the untuned model. This is mainly because the parameters was tuned considering knowledge about the nature of the data (e.g. rbf was used and his parameters used in GridSearch because there is some papers that shows rbf's performance to binary classification). The training time was a few times bigger because the GridSearch run for the parameters.

The table below compare them:

| Model             | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| Untuned (size=300)|      0.0107s            |      0.0025s           |    0.9074        |    0.8169       |
| Tuned             |      0.9743s            |      0.0028s           |    0.9542        |    0.8267       |
| Tuned (tips)      |      4.6517s            |      0.0021s           |    0.8532        |    0.8138       |

obs: I've used the tips from the reviwer to normalize and use np.logspace in C and gamma but found worst result. Actually I can't reproduce the model to get the same result as before the tip update. Why is that?

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.